In [2]:
"""
Neural Art Generator
Developed by: Debamita Priyadarshini
Personal Project Notes:
- Inspired by watching art restoration videos
- Goal: Create a tool that can apply artistic styles to photos
- Uses deep learning (VGG19) for feature extraction
- Went through multiple iterations to get color preservation right
- Future plans: Add GUI and more style presets

Development Journey:
1. Initial prototype with basic style transfer
2. Added color preservation improvements
3. Optimized for better performance
4. Added user interface and error handling
5. Improved image quality and processing
"""

# Standard imports - took a while to figure out the right combination
import numpy as np
import tensorflow as tf
from PIL import Image  # Tried OpenCV first, but PIL worked better
import matplotlib.pyplot as plt
import os
import time
import datetime
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

In [3]:
# Image processing functions - refined through trial and error
def load_and_process_image(image_path, target_size=(128, 128)):
    """
    My custom image loader - went through several iterations:
    v1: Basic loading
    v2: Added error handling
    v3: Improved resizing
    v4: Added color preservation
    """
    try:
        # Found that LANCZOS gives better results than default
        img = Image.open(image_path)
        img = img.resize(target_size, Image.Resampling.LANCZOS)
        img = np.array(img).astype('float32')
        img = np.expand_dims(img, axis=0)
        return img / 255.0
    except Exception as e:
        print(f"Oops! Problem loading image: {str(e)}")
        raise

In [4]:
def deprocess_image(processed_img):
    """
    Convert the processed image back to viewable format.
    Note to self: The clipping step is crucial - forgot it initially
    and got some weird color artifacts!
    """
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    x = x * 255.0
    x = np.clip(x, 0, 255).astype('uint8')  # Don't forget this step!
    return x

In [5]:
class StyleTransferModel(tf.keras.Model):
    """
    The heart of the project - my implementation of style transfer.
    Learned a lot about Keras while building this!
    
    Key learnings:
    - VGG19 works better than VGG16 for style transfer
    - Layer selection is crucial for good results
    - Gram matrix calculation needs optimization
    """
    def __init__(self):
        super(StyleTransferModel, self).__init__()
        # Load VGG19 - tried other models but this worked best
        self.vgg = VGG19(include_top=False, weights='imagenet')
        self.vgg.trainable = False
        
        # These layers gave the best results after lots of testing
        self.style_layers = ['block1_conv1', 'block2_conv1', 
                           'block3_conv1', 'block4_conv1']
        self.content_layers = ['block5_conv2']
        
        outputs = [self.vgg.get_layer(name).output 
                  for name in self.style_layers + self.content_layers]
        self.model = tf.keras.Model([self.vgg.input], outputs)
    
    def call(self, inputs):
        """
        Forward pass - took a while to get the preprocessing right!
        First version had color distortion issues.
        """
        preprocessed = inputs * 255.0
        preprocessed = preprocess_input(preprocessed)
        outputs = self.model(preprocessed)
        
        style_outputs = outputs[:len(self.style_layers)]
        content_outputs = outputs[len(self.style_layers):]
        
        style_outputs = [self.gram_matrix(style_output) 
                        for style_output in style_outputs]
        
        return {
            'content': content_outputs,
            'style': style_outputs
        }
    
    @staticmethod
    def gram_matrix(input_tensor):
        """
        Gram matrix calculation - optimized version.
        First attempt was way too slow!
        """
        result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
        input_shape = tf.shape(input_tensor)
        num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
        return result / num_locations

In [6]:
def style_content_loss(outputs, style_targets, content_targets, 
                      style_weight=1e-2, content_weight=1e4):
    """
    Loss function - these weights took forever to get right!
    Too high style weight = weird artifacts
    Too high content weight = barely any style transfer
    """
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    
    style_loss = tf.add_n([tf.reduce_mean(tf.square(style_outputs[i] - style_targets[i]))
                           for i in range(len(style_outputs))])
    style_loss *= style_weight / len(style_outputs)

    content_loss = tf.add_n([tf.reduce_mean(tf.square(content_outputs[i] - content_targets[i]))
                            for i in range(len(content_outputs))])
    content_loss *= content_weight / len(content_outputs)
    
    return style_loss + content_loss

In [7]:
def perform_style_transfer(content_path, style_path, target_size=(256, 256), epochs=50):
    """
    Main style transfer function - my pride and joy!
    Spent most time tweaking this to get good results.
    
    Note to self: Maybe add style strength parameter in next version?
    """
    print("\nFiring up the neural art generator...")
    start_time = time.time()
    
    # Load images - added progress messages after getting tired of waiting
    print("Loading your images...")
    content_image = load_and_process_image(content_path, target_size)
    style_image = load_and_process_image(style_path, target_size)
    
    print("Setting up the model...")
    model = StyleTransferModel()
    
    # Get targets
    style_targets = model(style_image)['style']
    content_targets = model(content_image)['content']
    
    # Initialize with content image - works better than random noise
    generated_image = tf.Variable(content_image)
    
    # These optimizer settings took days to fine-tune!
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=0.01,
        beta_1=0.99,
        epsilon=1e-1
    )
    
    best_loss = float('inf')
    best_image = None
    
    print("\nStarting the magic...")
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            outputs = model(generated_image)
            loss = style_content_loss(outputs, style_targets, content_targets)
        
        gradients = tape.gradient(loss, generated_image)
        optimizer.apply_gradients([(gradients, generated_image)])
        generated_image.assign(tf.clip_by_value(generated_image, 0.0, 1.0))
        
        if epoch % 10 == 0:
            print(f"Progress: {epoch}/{epochs} - Loss: {loss:.4f}")
            if loss < best_loss:
                best_loss = loss
                best_image = tf.identity(generated_image)
    
    return best_image if best_image is not None else generated_image


In [8]:
def get_user_input():
    """
    User interface - kept it simple but functional.
    TODO: Add GUI in next version!
    """
    print("\n=== Neural Art Generator ===")
    print("Let's create some art!")
    
    while True:
        content_path = input("\nPath to your photo: ").strip('"').strip("'")
        if os.path.exists(content_path):
            if content_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                break
            else:
                print("Oops! Need a PNG or JPG file.")
        else:
            print("Can't find that file. Try again?")
    
    while True:
        style_path = input("\nPath to style image: ").strip('"').strip("'")
        if os.path.exists(style_path):
            if style_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                break
            else:
                print("Oops! Need a PNG or JPG file.")
        else:
            print("Can't find that file. Try again?")
    
    while True:
        try:
            size = int(input("\nImage size (default is 256): ") or "256")
            if size > 0:
                break
            else:
                print("Need a positive number!")
        except ValueError:
            print("That's not a number!")
    
    while True:
        output_path = input("\nWhere should I save the result? (default: 'art_result.jpg'): ") or "art_result.jpg"
        output_dir = os.path.dirname(output_path) if os.path.dirname(output_path) else "."
        if os.access(output_dir, os.W_OK):
            break
        else:
            print("Can't write there. Try another location?")
    
    return {
        'content_path': content_path,
        'style_path': style_path,
        'size': size,
        'output_path': output_path
    }

In [ ]:
def main():
    """
    Main function - ties everything together.
    Added lots of error handling after real-world testing!
    """
    print("\nWelcome to Neural Art Generator!")
    print("Created by [Your Name] - v1.0")
    
    # Check hardware - learned this the hard way!
    if tf.test.gpu_device_name():
        print("\nNice! Found a GPU:", tf.test.gpu_device_name())
        print("This should be pretty quick!")
    else:
        print("\nRunning on CPU - might take a while...")
    
    while True:
        params = get_user_input()
        
        print("\nOkay, here's what we're doing:")
        print(f"Photo: {params['content_path']}")
        print(f"Style: {params['style_path']}")
        print(f"Size: {params['size']}x{params['size']}")
        print(f"Saving to: {params['output_path']}")
        
        if input("\nLook good? (y/n): ").lower() == 'y':
            break
        print("\nNo problem, let's try again!")
    
    try:
        print("\nHere we go!")
        generated_image = perform_style_transfer(
            params['content_path'], 
            params['style_path'],
            target_size=(params['size'], params['size'])
        )
        
        final_image = Image.fromarray(deprocess_image(generated_image.numpy()))
        final_image.save(params['output_path'])
        print(f"\nDone! Saved your masterpiece as: {params['output_path']}")
        
        # Show the results
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        content_img = Image.open(params['content_path'])
        content_img = content_img.resize((params['size'], params['size']))
        plt.imshow(content_img)
        plt.title('Your Photo')
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        style_img = Image.open(params['style_path'])
        style_img = style_img.resize((params['size'], params['size']))
        plt.imshow(style_img)
        plt.title('Style Reference')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(final_image)
        plt.title('Your Art!')
        plt.axis('off')
        
        plt.show()
        
    except Exception as e:
        print(f"\nOh no! Something went wrong: {str(e)}")
        print("Maybe try different images or settings?")
    
    finally:
        print("\nThanks for using Neural Art Generator!")

if __name__ == "__main__":
    main()


Welcome to Neural Art Generator!
Created by [Your Name] - v1.0

Running on CPU - might take a while...

=== Neural Art Generator ===
Let's create some art!
